# **MODIS vs THAAO comparison of Integrated Water Vapor (IWV)**
[MODIS](https://modis.gsfc.nasa.gov/about/specifications.php): Moderate Resolution Imaging Spectroradiometer  <br>
[THAAO](https://www.thuleatmos-it.it/index.php): Thule High Arctic Atmospheric Observatory, Greenland  (coordinates  __76.5145, -68.7432__, altitude a.s.l. __220 m__)  <br>

__MXD05_L2 IWV product__ <br>
_References_ <br>
[MYD05_L2](https://atmosphere-imager.gsfc.nasa.gov/products/water-vapor) <br>

_Algorithm Theoretical Basis Documents (ATBDs)_ <br>
[ATBD](https://atmosphere-imager.gsfc.nasa.gov/documentation/atbds-plans-guides) <br>
[The MODIS Near-IR Water Vapor Algorithm](https://atmosphere-imager.gsfc.nasa.gov/sites/default/files/ModAtmo/atbd_mod03_0.pdf) <br>
[MODIS Atmospheric Profile Retrieval
ATBD MOD07 Collection 6](https://modis-images.gsfc.nasa.gov/_docs/MOD07_atbd_v7_April2011.pdf) <br>
[Atmospheric Correction Algorithm: Spectral
Reflectances MOD08](https://modis.gsfc.nasa.gov/data/atbd/atbd_mod08.pdf) <br>
[MODIS Atmosphere L3 Gridded Product
(ATBD) & Users Guide](https://atmosphere-imager.gsfc.nasa.gov/sites/default/files/ModAtmo/L3_ATBD_C6_C61_2019_02_20.pdf) <br>

__THAAO IWV product__ <br>
_References_ <br>
[The water Vapor Emission Spectrometer for Polar Atmospheres at 22 GHz](https://www.thuleatmos-it.it/data/vespa22/index.php)<br> 
[Integrated Water Vapor](https://www.thuleatmos-it.it/dataaccess/vapor/index.php)<br>
 

In [ ]:
%run reader.ipynb

## __MXD05_L2 and THAAO IWV products__ 

Satellite MODIS MXD05_L2 data products covering the Greenland region on a time interval spanning from July 2016 to January 2022 were first downloaded from the LAADS repository by means of custom Python scripts. Timeseries of IWV IR and ancillary data products were then extracted on a point geometry centered on the THAAO Observatory, Greenland from the MXD05_L2 products and packed together as csv files for further analisys. <br>

Thule (Greenland) IWV data timeseries have been downloaded from the [Integrated Water Vapor](https://www.thuleatmos-it.it/dataaccess/vapor/index.php) THAAO data access facility were the data are freely accessible. Likewise MODIS satellite IWV product, these data contain vertical column integrated amount  water vapor or precipitable water retrieved by means of the water Vapor Emission Spectrometer for Polar Atmospheres at 22 GHz (VESPA-22).  <br> 


## __Loading MODIS and VESPA timeseries__  
IWV timeseries of MODIS in .csv and THAAO-VESPA-22 in .txt formats are read into pandas dataframes. <br>
MODIS Near-IR product has an estimated additional 13% error column derived from the maximum error claimed in the NearIR IWV product ATBD (see sect. 3.1.4, page 16). <br>
THAAO IWV data are given in $kg/m^2$ units and thus divided for factor of 10 to have homogeoeous units with MODIS IWV data that are given in $cm$. <br>  

In [ ]:
mod_data = modis_read('MODIS/MOD05_5km_totale.csv')
myd_data = modis_read('MODIS/MYD05_5km_totale.csv')
vespa_data = vespa_read('VESPA/Vapor_20160716_20220311.txt')

# **MODIS data selection by max IWV and distance from THAAO**

### **Set max IWV and distance from THAAO**

In [ ]:
max_IWV = 50.              # [mm]
max_dist_from_THAAO = 10.  # [km]

### **MODIS data selection**

In [ ]:
mod_data = df_drop(mod_data, mod_data.IWV_MODIS > max_IWV)
myd_data = df_drop(myd_data, myd_data.IWV_MODIS > max_IWV )

In [ ]:
mod_data = distance_from_station(mod_data,drop_above=max_dist_from_THAAO)
myd_data = distance_from_station(myd_data,drop_above=max_dist_from_THAAO)

#### **Controllo numero di punti mod, myd e mxd per max IWV e distanza da THAAO selezionati**

In [ ]:
number_mod_data_p = len(mod_data.IWV_MODIS) 
number_myd_data_p = len(myd_data.IWV_MODIS)
number_mxd_data_points = number_mod_data_p + number_myd_data_p

In [ ]:
print(f'max IWV = {max_IWV} [mm]')
print(f'max distance from THAOO = {max_dist_from_THAAO} [km]\n')
print(f'Total number of MOD data points selected =    {number_mod_data_p}')
print(f'Total number of MYD data points selected =    {number_myd_data_p}\n')
print(f'Total number of MODIS data points selected = {number_mxd_data_points}\n')

#### **Controllo distanza e lat,lon dei pixel MODIS dal THAAO**

In [ ]:
print(f'MOD distance [km]:  min = {mod_data.diff_distance.min()}   max = {mod_data.diff_distance.max()}')
print(f'MYD distance [km]:  min = {myd_data.diff_distance.min()}   max = {myd_data.diff_distance.max()}\n')
print(f'MOD latitude  :  min = {mod_data.latitude.min()}    max = {mod_data.latitude.max()}')
print(f'MOD longitude :  min = {mod_data.longitude.min()}   max = {mod_data.longitude.max()} \n')
print(f'MYD latitude  :  min = {myd_data.latitude.min()}    max = {myd_data.latitude.max()}')
print(f'MYD longitude :  min = {myd_data.longitude.min()}   max = {myd_data.longitude.max()} \n')

#### **Numero punti per mese** 

In [ ]:
mod_data.Time.groupby([mod_data['Time'].dt.month.rename('month')]).agg({'count'}).transpose()

In [ ]:
myd_data.Time.groupby([myd_data['Time'].dt.month.rename('month')]).agg({'count'}).transpose()

## **Other sensors**

## **MODIS and THAAO Integrated Water Vapor (IWV) total timeseries**
Simple linear plots of the whole MODIS and THAAO IVW timeseries.

#### **Visualization options**

In [ ]:
width = 1100
height = 300 
size = 3
scale = 0.3
color_MOD = 'cyan'
color_MYD = '#F2583E'
color_THAAO = 'green'

alpha=100
dynspread=False
datashade=False
legend='top_right'

# text options
xlabel='Time'
ylabel='IWV    [mm]'

plot_defaults = dict(xlabel=xlabel, ylabel=ylabel,height=height,
                width=width,alpha=alpha,dynspread=dynspread,
                datashade=datashade,size=size, legend=legend)

### **Time-series with MODIS data selected by max IWV and distance from THAAO**

In [ ]:
mod_timeseries = mod_data.IWV_MODIS.hvplot.scatter(color=color_MOD, label='MOD',
                                                 title='IWV MOD',
                                                 **plot_defaults)
myd_timeseries = myd_data.IWV_MODIS.hvplot.scatter(color=color_MYD, label='MYD',
                                                 title='IWV MYD',
                                                 **plot_defaults)

In [ ]:
thaao_timeseries = vespa_data.IWV_THAAO.hvplot.scatter(color=color_THAAO, label='VESPA',
                                                       fields={'date': 'Date'},
                                                       title='IWV THAAO - VESPA', 
                                                       **plot_defaults)

### **Time-series plots with errors**

### **Time-series plots**

In [ ]:
((mod_timeseries*myd_timeseries).opts(title='MXD') 
 + thaao_timeseries).cols(1)

### **Time-series plots of all MODIS and VESPA-22 IWV data on the same time reference** <br>

In [ ]:
(mod_timeseries*myd_timeseries * thaao_timeseries).opts(title=title_mxd, show_grid=True)

### __QA__ 
__TODO: inserire il discorso sul check dei Quality Assessment: tutti i dati sono best quality__ <br>

# __Merging pandas dataframes to filter MODIS and THAAO data points closest in time__

__TODO: rivedere questo discorso, troppo semplice e contorto insieme__ <br>
Due to time lenght of the timeseries, even if the notebook representation allows to zoom in into the datasets, it is still not easy to see how close in time two data points are. Since it makes little sense to compare two MODIS and THAAO IWV data points if they are too far apart, and they can easily be hours or days far apart, therefore easily mistaken if one relies solely on the visual inspection of the plots, a tool to select and compare only the data points which are closer in time is required. The following code allows to filter the data on a timespan basis by suitably merging modis and thaao pandas dataframes while defining the Timedelta parameter (expressed in s,m,d, i.e. seconds, minutes or days respectively) that allows to operate the time-based filter. Different choices of Timedelta allow to widen or narrow the time span around each MODIS data point and select for the comparison only the THAAO data points that fall within the chosen interval. This selection is based on MODIS data points because THAAO measures are far more dense. <br>  

In [ ]:
# processing and visualization parameters

## **Set max time difference (deltatime) between MODIS and THAAO measures**

In [ ]:
deltatime = '10m'

In [ ]:
new_myd_vespa = closest_in_time2(myd_data,vespa_data,'IWV_MODIS','IWV_THAAO',on='Time',delta_time=deltatime)
new_mod_vespa = closest_in_time2(mod_data,vespa_data,'IWV_MODIS','IWV_THAAO',on='Time',delta_time=deltatime)

#### **Controllo numero di punti mod, myd e mxd per il deltatime selezionato**

In [ ]:
num_mod_data_p_Dt = len(new_mod_vespa.IWV_MODIS) 
num_myd_data_p_Dt = len(new_myd_vespa.IWV_MODIS)
num_mxd_data_p_Dt = num_mod_data_p_Dt + num_myd_data_p_Dt

In [ ]:
print(f'Number of data points selected with deltatime = {deltatime}in')
print(f'within a max distance from THAOO = {max_dist_from_THAAO} km\n')
print(f'Selected number of MOD data points =          {num_mod_data_p_Dt}')
print(f'Selected number of MYD data points =          {num_myd_data_p_Dt}\n')
print(f'Total selected number of MODIS data points =  {num_mxd_data_p_Dt}\n')

#### **Controllo distanza e lat,lon dei pixel MODIS dal THAAO**

In [ ]:
print(f'MOD distance [km]:  min = {new_mod_vespa.diff_distance.min()}   max = {new_mod_vespa.diff_distance.max()}')
print(f'MYD distance [km]:  min = {new_myd_vespa.diff_distance.min()}   max = {new_myd_vespa.diff_distance.max()}\n')
print(f'MOD latitude  :  min = {new_mod_vespa.latitude.min()}    max = {new_mod_vespa.latitude.max()}')
print(f'MOD longitude :  min = {new_mod_vespa.longitude.min()}   max = {new_mod_vespa.longitude.max()} \n')
print(f'MYD latitude  :  min = {new_myd_vespa.latitude.min()}    max = {new_myd_vespa.latitude.max()}')
print(f'MYD longitude :  min = {new_myd_vespa.longitude.min()}   max = {new_myd_vespa.longitude.max()} \n')

#### **Numero punti per mese** 

In [ ]:
new_mod_vespa.Time.groupby([new_mod_vespa['Time'].dt.month.rename('month')]).agg({'count'}).transpose()

In [ ]:
new_myd_vespa.Time.groupby([new_myd_vespa['Time'].dt.month.rename('month')]).agg({'count'}).transpose()

#### **Controllo dei pixel selezionati con deltatime, max and min time difference tra le misure MODIS e VESPA**
TODO

#### **Controllo VZA mod e myd per il deltatime selezionato**

In [ ]:
print(f'MOD View Zenith Angle:  min = {new_mod_vespa.vza.min()}   max = {new_mod_vespa.vza.max()}')
print(f'MYD View Zenith Angle:  min = {new_myd_vespa.vza.min()}   max = {new_myd_vespa.vza.max()}\n')

### **New time-series definition after time range selection**

In [ ]:
t_mod_vespa=f'IWV MOD - VESPA{chr(0x20)*50}deltatime={deltatime}{chr(0x20)*50}# MOD data = {num_mod_data_p_Dt}'
t_myd_vespa=f'IWV MYD - VESPA{chr(0x20)*50}deltatime={deltatime}{chr(0x20)*50}# MYD data = {num_myd_data_p_Dt}'
t_mxd_vespa=f'IWV MXD - VESPA{chr(0x20)*50}deltatime={deltatime}{chr(0x20)*50}# MXD data = {num_mxd_data_p_Dt}'

new_mod_vespa_ts = new_mod_vespa.IWV_MODIS.hvplot.scatter(x='Time_IWV_MODIS',
                                                          color=color_MOD, 
                                                          label='MOD',
                                                          title=t_mod_vespa,
                                                          **plot_defaults)
new_myd_vespa_ts = new_myd_vespa.IWV_MODIS.hvplot.scatter(x='Time_IWV_MODIS',
                                                          color=color_MYD, 
                                                          label='MYD',
                                                          title=t_myd_vespa,
                                                          **plot_defaults)

new_vespa_mod_ts = new_mod_vespa.IWV_THAAO.hvplot.scatter(x='Time_IWV_THAAO',
                                                         color=color_THAAO,
                                                         label='VESPA',                                                        
                                                         **plot_defaults)
new_vespa_myd_ts = new_myd_vespa.IWV_THAAO.hvplot.scatter(x='Time_IWV_THAAO',
                                                         color=color_THAAO,
                                                         label='VESPA',                                                        
                                                         **plot_defaults)

In [ ]:
new_mxd_vespa = pd.concat([new_mod_vespa,new_myd_vespa], sort=True)

### **New time-series plots after time range selection** 

In [ ]:
(new_mod_vespa_ts * new_myd_vespa_ts
* new_vespa_mod_ts * new_vespa_myd_ts).opts(title=t_mxd_vespa, show_grid=True)

# **Data Analisys MODIS vs VESPA-22**

### Regressione con Holoviews

### **Regressione con sklearn**

In [ ]:
regress_mod_sk = regression_sklearn(new_mod_vespa.IWV_MODIS,new_mod_vespa.IWV_THAAO)
regress_mod_sk_text = regression_eq_text(new_mod_vespa.IWV_MODIS,
                                         new_mod_vespa.IWV_THAAO, regress_mod_sk)
regress_myd_sk = regression_sklearn(new_myd_vespa.IWV_MODIS,
                                    new_myd_vespa.IWV_THAAO, color='black')
regress_myd_sk_text = regression_eq_text(new_myd_vespa.IWV_MODIS,
                                         new_myd_vespa.IWV_THAAO, regress_myd_sk)
regress_mxd_vespa_sk = regression_sklearn(new_mxd_vespa.IWV_MODIS,new_mxd_vespa.IWV_THAAO, 
                                    color='cyan')
regress_mxd_vespa_sk_text = regression_eq_text(new_mxd_vespa.IWV_MODIS,
                                         new_mxd_vespa.IWV_THAAO, regress_mxd_vespa_sk)

#### **Bisettrice**

In [ ]:
slope = hv.Slope(1, 0).opts(line_color='red')   # generic bisector

## **MODIS vs THAAO Integrated Water Vapor (IWV) timeseries scatter plots**

## **Basic scatter plots**

#### **Visualization options: scatter plots**

In [ ]:
# line-dots colors                
color_MOD = 'cyan'
color_MYD = '#F2583E'
color_MXD = 'magenta'
color_THAAO = 'green'

# general default settings
x = ''
y = ''
xlabel = 'IWV MOD    [mm]' 
ylabel = 'IWV VESPA    [mm]'
color = ''
label = ''
height = 500
width = 600
c = ''
s = ''
cmap = ''
alpha = 10
#scale = 1
#size = 2
legend = 'top_right'
dynspread = False
datashade = False  # datashading not available, può essere tolto
grid=True
title = ''
 
# ATTENZIONE nell'uso di **kwargs (come splot_defaults): 
# - il parametro opzionale s che viene passato come kwds  
#   NON può essere definito se non vieno utilizzato:
#   le celle non girano. 
#   Va definito dopo con splot_defaults.update(s='colonna_df'), 
#   SOLO quando si usa
# - il parametro size blocca l'effetto del parametro s 
#   (i punti hanno dim bloccata a size che non varia con s)

splot_defaults = dict(x=x, y=x,
                      xlabel=xlabel, ylabel=ylabel,
                      color=color, label=label,
                      height=height, width=width,
                      c=c,
                      cmap=cmap, 
                      alpha=alpha,
                      scale=scale, 
                      legend=legend,
                      dynspread=dynspread,datashade=datashade,
                      grid=grid,
                      title = title)

## come cambiare i vaori delle keywords definite o aggiungerne altri
## metodo 1
#print(splot_defaults)
#s_='diff_int'
#changed_plot_values = dict(s=s_)
#splot_defaults.update(changed_plot_values)
#print(f' metodo 1: {splot_defaults}')
#
## metodo 2
#s_=''
#splot_defaults.update(s=s)
#print(f' metodo 2: {splot_defaults}')
print(splot_defaults)

### **Basic scatter plots variables and visual kwargs options**

In [ ]:
splot_defaults.update(x='IWV_MODIS', y='IWV_THAAO', 
                      xlabel = 'IWV MOD    [mm]', 
                      ylabel = 'IWV VESPA    [mm]')
print(f' xxxxxxx: {splot_defaults}')
splot_defaults.update(color=color_MOD, label='MOD')
vespaVSmod = new_mod_vespa.hvplot.scatter(**splot_defaults)

splot_defaults.update(color=color_MYD, label='MYD')
vespaVSmyd = new_myd_vespa.hvplot.scatter(**splot_defaults)

splot_defaults.update(color=color_MXD, label='MXD')
vespaVSmxd = new_mxd_vespa.hvplot.scatter(**splot_defaults)

In [ ]:
# MXD: MOD e MYD insieme (colori diversi, dataset mod e myd diversi)
(vespaVSmod*vespaVSmyd*slope*regress_mxd_vespa_sk_text
* regress_mod_sk[0]*regress_myd_sk[0]*regress_mxd_vespa_sk[0]
  ).opts(xlabel='IWV MXD    [mm]', title=t_mxd_vespa, show_grid=True)

### **Diff (seconds) scatter plots variables and visual options**

In [ ]:
# Difference in time (seconds) between MODIS and VESPAdata points
splot_defaults.update(c='vza', s='diff_int')

splot_defaults.update(#c='diff_int', s='diff_int', 
                      #scale = 10, alpha = 0.5, size=1,
                      alpha = 0.5, 
                      color=color_MOD, label='MOD')
vespaVSmod = new_mod_vespa.hvplot.scatter(**splot_defaults
                                         ).hist(num_bins=21, dimension=['IWV_MODIS','IWV_THAAO'])

splot_defaults.update(#c='diff_int', s='diff_int', 
                      scale = 10, alpha = 0.5, size=1,
                      color=color_MYD, label='MYD')
vespaVSmyd = new_myd_vespa.hvplot.scatter(**splot_defaults
                                         ).hist(num_bins=21, dimension=['IWV_MODIS','IWV_THAAO'])

splot_defaults.update(#c='diff_int', s='diff_int', 
                      scale = 10, alpha = 0.5, size=1,
                      color=color_MXD, label='MXD')
vespaVSmxd = new_mxd_vespa.hvplot.scatter(**splot_defaults
                                         ).hist(num_bins=21, dimension=['IWV_MODIS','IWV_THAAO'])

In [ ]:
# MXD: MOD e MYD insieme (colori diversi, dataset mod e myd diversi)
(vespaVSmod*vespaVSmyd*slope*regress_mxd_vespa_sk_text
*regress_mod_sk[0]*regress_myd_sk[0]*regress_mxd_vespa_sk[0])

## Scatter dimensione punti su differenza secondi

In [ ]:
scale = 0.5
alpha = 0.7
vespaVSmod = new_mod_vespa.hvplot.scatter(x='IWV_MODIS', y='IWV_THAAO',
                                      height=500, width=600, c='diff_int',
                                      s='diff_int', scale=scale,
                                      alpha=alpha,
                                      xlabel='IWV MOD    [mm]', 
                                      ylabel='IWV THAAO    [mm]'
                                     ).hist(num_bins=21,dimension=['IWV_MODIS','IWV_THAAO'])


vespaVSmyd = new_myd_vespa.hvplot.scatter(x='IWV_MODIS', y='IWV_THAAO', 
                                      height=500, width=600, c='diff_int',
                                      s='diff_int', scale=scale, 
                                      alpha=alpha,
                                      xlabel='IWV MYD    [mm]', 
                                      ylabel='IWV THAAO    [mm]'
                                     ).hist(num_bins=21,dimension=['IWV_MODIS','IWV_THAAO'])

(vespaVSmod*slope*regress_mod_sk[0]*regress_mod_sk_text
+ vespaVSmyd*slope*regress_myd_sk[0]*regress_myd_sk_text
+ vespaVSmod*vespaVSmyd*slope*regress_mxd_vespa_sk_text
* regress_mod_sk[0]*regress_myd_sk[0]*regress_mxd_vespa_sk[0]).cols(1)

### Scatter dimensione punti su differenza distanza geografica

In [ ]:
# diff_distance non viene calcolata in closest_in_time2() perché varia poco e variano poco i risultati, 
# quindi la cella va in errore. In caso di verifica reinserire la colonna 'diff_distance' nel dataframe  

scale = 5
alpha = 0.7
vespaVSmod_scatter = new_mod_vespa.hvplot.scatter(x='IWV_MODIS', y='IWV_THAAO',
                                              height=500, width=600, c='vza',
                                              s='diff_distance', scale=scale,
                                              alpha=alpha,
                                              xlabel='IWV MOD    [mm]', 
                                              ylabel='IWV THAAO    [mm]'
                                             ).hist(num_bins=21,dimension=['IWV_MODIS','IWV_THAAO'])

vespaVSmyd_scatter = new_myd_vespa.hvplot.scatter(x='IWV_MODIS', y='IWV_THAAO',
                                              height=500, width=600, c='vza',
                                              s='diff_distance', scale=scale,
                                              alpha=alpha,
                                              xlabel='IWV MYD    [mm]',
                                              ylabel='IWV THAAO    [mm]'
                                             ).hist(num_bins=21,dimension=['IWV_MODIS','IWV_THAAO'])

(vespaVSmod_scatter*slope*regress_mod_sk[0]*regress_mod_sk_text
+ vespaVSmyd_scatter*slope*regress_myd_sk[0]*regress_myd_sk_text
+ vespaVSmod_scatter*vespaVSmyd_scatter*slope*regress_mxd_vespa_sk_text
 *regress_mod_sk[0]*regress_myd_sk[0]*regress_mxd_vespa_sk[0]).opts(width=400, height=300).cols(1)

### Scatter totale per piattaforma

In [ ]:
# diff_distance non viene calcolata in closest_in_time2() perché varia poco e variano poco i risultati, 
# quindi la cella va in errore. In caso di verifica reinserire la colonna 'diff_distance' nel dataframe  
scale = 5
vespaVSmod_scatter = new_mod_vespa.hvplot.scatter(x='IWV_MODIS', y='IWV_THAAO',
                                                  color=color_MOD,
                                              height=height, width=width, 
                                              c='vza',
                                              s='diff_distance', scale=scale,
                                              xlabel='IWV MOD    [mm]',
                                              ylabel='IWV THAAO    [mm]'
                                             ).hist(num_bins=21,dimension=['IWV_MODIS','IWV_THAAO'])


vespaVSmyd_scatter = new_myd_vespa.hvplot.scatter(x='IWV_MODIS', y='IWV_THAAO',
                                                  color=color_MYD,
                                              height=height, width=width, 
                                              c='vza',
                                              s='diff_distance', scale=scale, 
                                              xlabel='IWV MYD    [mm]', 
                                              ylabel='IWV THAAO    [mm]'
                                             ).hist(num_bins=21,dimension=['IWV_MODIS','IWV_THAAO'])

(vespaVSmod_scatter*slope*regress_mod_sk[0]*regress_mod_sk_text
+ vespaVSmyd_scatter*slope*regress_myd_sk[0]*regress_myd_sk_text
+ vespaVSmod_scatter*vespaVSmyd_scatter*slope*regress_mxd_vespa_sk_text
 *regress_mod_sk[0]*regress_myd_sk[0]*regress_mxd_vespa_sk[0]).cols(1)

In [ ]:
# ok s='diff_distance'
height=500
width=700
scale = 1
title = f'IWV MXD-VESPA              deltatime={deltatime}         # data points={num_mxd_data_p_Dt}'

vespaVSmod_scatter_ok = new_mod_vespa.hvplot.scatter(x='IWV_MODIS', y='IWV_THAAO',
                                              height=height, width=width, 
#                                              c='diff_distance',
                                              c='vza',
                                              s='diff_distance', 
                                              scale=scale,
                                              xlabel='IWV MOD    [mm]',
                                              ylabel='IWV VESPA    [mm]')

vespaVSmyd_scatter_ok = new_myd_vespa.hvplot.scatter(x='IWV_MODIS', y='IWV_THAAO',
                                              height=height, width=width, 
#                                              c='diff_distance',
                                              c='vza',
                                              s='diff_distance', 
                                              scale=scale,
                                              xlabel='IWV MYD    [mm]', 
                                              ylabel='IWV VESPA    [mm]')

(vespaVSmod_scatter_ok*vespaVSmyd_scatter_ok
*slope
*regress_mxd_vespa_sk_text
#*regress_MOD_sk[0]*regress_MYD_sk[0]
*regress_mxd_vespa_sk[0]).opts(title=title, show_grid=True, xlabel= 'IWV MXD    [mm]')

In [ ]:
# ok s='vza'
height=500
width=700
scale = 0.5
title = f'IWV MXD-VESPA              deltatime={deltatime}         # data points={num_mxd_data_p_Dt}'

vespaVSmod_scatter_ok = new_mod_vespa.hvplot.scatter(x='IWV_MODIS', y='IWV_THAAO',
                                              height=height, width=width, 
                                              c='vza',
                                              s='vza', scale=scale,
                                              xlabel='IWV MOD    [mm]',
                                              ylabel='IWV VESPA    [mm]')

vespaVSmyd_scatter_ok = new_myd_vespa.hvplot.scatter(x='IWV_MODIS', y='IWV_THAAO',
                                              height=height, width=width, 
                                              c='vza',
                                              s='vza', scale=scale, 
                                              xlabel='IWV MYD    [mm]', 
                                              ylabel='IWV VESPA    [mm]')

(vespaVSmod_scatter_ok*vespaVSmyd_scatter_ok
*slope
*regress_mxd_vespa_sk_text
#*regress_MOD_sk[0]*regress_MYD_sk[0]
*regress_mxd_vespa_sk[0]).opts(title=title, show_grid=True, xlabel= 'IWV MXD    [mm]')

### Stagionalità

In [ ]:
xlim = [0,2]
ylim = [0,2]
xlim = [0,25]
ylim = [0,25]
#x_offset=xlim[1]-30
#y_offset=ylim[0]+5
scale = 5

regress_mod_vespa_s_text = regression_eq_text(new_mod_vespa.IWV_MODIS,
                                        new_mod_vespa.IWV_THAAO, regress_mod_sk,
                                        x_offset=xlim[0]-0.5,y_offset=ylim[0]+0.5)
regress_myd_vespa_s_text = regression_eq_text(new_myd_vespa.IWV_MODIS,
                                        new_myd_vespa.IWV_THAAO, regress_myd_sk,
                                        x_offset=xlim[0]-0.5,y_offset=ylim[0]+0.5)
regress_mxd_vespa_s_text = regression_eq_text(new_mxd_vespa.IWV_MODIS,
                                        new_mxd_vespa.IWV_THAAO, regress_mxd_vespa_sk,
                                        x_offset=xlim[0]-0.5,y_offset=ylim[0]+0.5)

vespaVSmod_scatter = new_mod_vespa.hvplot.scatter(x='IWV_MODIS', y='IWV_THAAO', 
                                              height=height, width=width, 
                                              c='vza', 
                                              xlim=xlim, ylim=ylim, 
                                              s='diff_distance', scale=scale, 
                                              xlabel='IWV MOD    [mm]', 
                                              ylabel='IWV THAAO    [mm]',
                                              groupby=['Time.month'])

vespaVSmyd_scatter = new_myd_vespa.hvplot.scatter(x='IWV_MODIS', y='IWV_THAAO', 
                                              height=height, width=width, c='vza',
                                              xlim=xlim, ylim=ylim, 
                                              s='diff_distance', scale=scale, 
                                              xlabel='IWV MYD    [mm]',
                                              ylabel='IWV THAAO    [mm]',
                                              groupby=['Time.month'])

(vespaVSmod_scatter*slope*regress_mod_sk[0]*regress_mod_vespa_s_text
+ vespaVSmyd_scatter*slope*regress_myd_sk[0]*regress_myd_vespa_s_text
+ (vespaVSmod_scatter*vespaVSmyd_scatter*slope*regress_mxd_vespa_s_text
   *regress_mod_sk[0]*regress_myd_sk[0]*regress_mxd_vespa_sk[0]
  ).opts(title='DDDDD', xlabel= 'IWV MXD    [mm]')).cols(1)

### GroupBy month

In [ ]:
vespaVSmod_scatter = new_mod_vespa.hvplot.scatter(x='IWV_MODIS', y='IWV_THAAO',
                                              height=300,
                                              xlim=xlim, ylim=ylim,
                                              scale=0.3, xlabel='IWV MOD    [mm]',
                                              ylabel='IWV THAAO    [mm]',
                                              by=['Time.month'], groupby=['Time.month'])

vespaVSmyd_scatter = new_myd_vespa.hvplot.scatter(x='IWV_MODIS', y='IWV_THAAO',
                                              height=300, 
                                              xlim=xlim, ylim=ylim,
                                              scale=0.3, xlabel='IWV MYD    [mm]',
                                              ylabel='IWV THAAO    [mm]',
                                              by=['Time.month'], groupby=['Time.month'])

(vespaVSmod_scatter*slope*regress_mod_sk[0]*regress_mod_vespa_s_text
+ vespaVSmyd_scatter*slope*regress_myd_sk[0]*regress_myd_vespa_s_text
+ (vespaVSmod_scatter*vespaVSmyd_scatter*slope*regress_mod_sk[0]*regress_myd_sk[0]
*regress_mxd_vespa_sk[0]*regress_mxd_vespa_s_text
  ).opts(title='MXD vs VESPA IWV seasonality by month', 
#         xlabel = 'IWV MXD (mm)', ylabel='IWV THAAO (mm)')).cols(1)
         xlabel = 'IWV MXD    [mm]', ylabel='IWV THAAO    [mm]')).cols(1)

In [ ]:
vespaVSmod_scatter = new_mod_vespa.hvplot.scatter(x='IWV_MODIS', y='IWV_THAAO',
                                              height=300,
                                              xlim=xlim, ylim=ylim,
                                              scale=0.3, 
                                              xlabel='IWV MOD    [mm]', ylabel='IWV THAAO    [mm]',
                                              by=['Time.month'], groupby=['Time.month'])

vespaVSmyd_scatter = new_myd_vespa.hvplot.scatter(x='IWV_MODIS', y='IWV_THAAO',
                                              height=300, 
                                              xlim=xlim, ylim=ylim,
                                              scale=0.3,
                                              xlabel='IWV MOD    [mm]', ylabel='IWV THAAO    [mm]',
                                              by=['Time.month'], groupby=['Time.month'])

pn.panel((vespaVSmod_scatter*slope*regress_mod_sk[0]*regress_mod_vespa_s_text
+ vespaVSmyd_scatter*slope*regress_myd_sk[0]*regress_myd_vespa_s_text
+ (vespaVSmod_scatter*vespaVSmyd_scatter*slope*regress_mod_sk[0]*regress_myd_sk[0]
*regress_mxd_vespa_sk[0]*regress_mxd_vespa_s_text
  ).opts(title='MXD vs VESPA IWV seasonality animation', 
         xlabel = 'IWV MXD    [mm]', ylabel='IWV THAAO    [mm]')).cols(1),
#         xlabel = 'IWV MXD (mm)', ylabel='IWV THAAO (mm)')).cols(1),
         widget_type='scrubber', widget_location='top')

### By month

In [ ]:
#cmap = 'cet_CET_C5'
vespaVSmod_scatter = new_mod_vespa.hvplot.scatter(x='IWV_MODIS', y='IWV_THAAO',
                                               height=height, width=width,
                                              cmap=cmap,
                                              scale=0.3, 
                                              xlabel='IWV MOD    [mm]', ylabel='IWV THAAO    [mm]',
                                              by=['Time.month'])

vespaVSmyd_scatter = new_myd_vespa.hvplot.scatter(x='IWV_MODIS', y='IWV_THAAO',
                                              height=height, width=width,
                                              cmap=cmap,
                                              scale=0.3, 
                                              xlabel='IWV MOD    [mm]', ylabel='IWV THAAO    [mm]',
                                              by=['Time.month'])

(vespaVSmod_scatter*slope*regress_mod_sk[0]*regress_mod_vespa_s_text
+ vespaVSmyd_scatter*slope*regress_myd_sk[0]*regress_myd_vespa_s_text
+ (vespaVSmod_scatter*vespaVSmyd_scatter*slope*regress_mod_sk[0]*regress_myd_sk[0]
*regress_mxd_vespa_sk[0]*regress_mxd_vespa_sk_text 
  ).opts(title='MXD vs VESPA IWV seasonality', 
         xlabel = 'IWV MXD   [mm]', ylabel='IWV THAAO   [mm]')).cols(1)

In [ ]:
# ok stagionalità
#cmap = 'hsv'
vespaVSmod_scatter = new_mod_vespa.hvplot.scatter(x='IWV_MODIS', y='IWV_THAAO',
                                               height=height, width=width,
#                                              cmap=cmap,
                                              scale=0.3, 
                                              xlabel='IWV MOD    [mm]', ylabel='IWV VESPA    [mm]',
                                              by=['Time.month'])

vespaVSmyd_scatter = new_myd_vespa.hvplot.scatter(x='IWV_MODIS', y='IWV_THAAO',
                                              height=height, width=width,
#                                              cmap=cmap,
                                              scale=0.3, 
                                              xlabel='IWV MOD    [mm]', ylabel='IWV VESPA    [mm]',
                                              by=['Time.month'])


(vespaVSmod_scatter*vespaVSmyd_scatter
# *slope
#*regress_MOD_sk[0]*regress_MYD_sk[0]
*regress_mxd_vespa_sk[0]*regress_mxd_vespa_sk_text
).opts(title='IWV seasonality', 
       xlabel = 'IWV MXD    [mm]', ylabel='IWV THAAO    [mm]', show_grid=True)

In [ ]:
scale = 5
vespaVSmod_scatter = new_mod_vespa.hvplot.scatter(x='vza', y='diff_perc_IWV',
                                              height=height, width=width, 
#                                              c='vza',
#                                              s='diff_distance', scale=scale,
                                              xlabel='vza',
                                              ylabel='difference IWV THAAO'
                                             ) #.hist(num_bins=21,dimension=['IWV_MODIS','IWV_THAAO'])


vespaVSmyd_scatter = new_myd_vespa.hvplot.scatter(x='vza', y='diff_perc_IWV',
                                              height=height, width=width, 
#                                              c='vza',
#                                              s='diff_distance', scale=scale, 
                                              xlabel='vza', 
                                              ylabel='difference IWV THAAO'
                                             ) #.hist(num_bins=21,dimension=['IWV_MODIS','IWV_THAAO'])

#(vespaVSmod_scatter*slope*regress_mod_sk[0]*regress_mod_sk_text
#+ vespaVSmyd_scatter*slope*regress_myd_sk[0]*regress_myd_sk_text
#+ vespaVSmod_scatter*vespaVSmyd_scatter*slope*regress_mxd_vespa_sk_text
# *regress_mod_sk[0]*regress_myd_sk[0]*regress_mxd_vespa_sk[0]).cols(1)
(vespaVSmod_scatter
+ vespaVSmyd_scatter).cols(1)

### Diff IWV MXD-THAAO vs IWV THAAO 

In [ ]:
height=380
width=1000

In [ ]:
new_mod_vespa.head(1)

In [ ]:
Diff_iwv_vespaVSmod_scatter = new_mod_vespa.hvplot.scatter(x='IWV_THAAO',
#                                                   y='diff_IWV', by=['time.month'],
#                                                   xlabel='IWV VESPA    [cm]',
#                                                   ylabel='Diff MOD-VESPA    [cm]',
                                                   y='diff_IWV', by=['Time.month'],
                                                   xlabel='IWV VESPA    [mm]',
                                                   ylabel='Diff MOD-VESPA    [mm]',
                                                   height=height, width=width, 
                                                   scale=scale,grid=True)

Diff_iwv_vespaVSmyd_scatter = new_myd_vespa.hvplot.scatter(x='IWV_THAAO',
#                                                   y='diff_IWV', by=['time.month'],
#                                                   xlabel='IWV VESPA    [cm]',
#                                                   ylabel='Diff MYD-VESPA    [cm]',
                                                   y='diff_IWV', by=['Time.month'],
                                                   xlabel='IWV VESPA    [mm]',
                                                   ylabel='Diff MYD-VESPA    [mm]',
                                                   height=height, width=width, 
                                                   scale=scale,grid=True)

(Diff_iwv_vespaVSmod_scatter
+ Diff_iwv_vespaVSmyd_scatter
+ (Diff_iwv_vespaVSmod_scatter*Diff_iwv_vespaVSmyd_scatter
  ).opts(ylabel='Diff MXD-VESPA    [mm]')).cols(1)

In [ ]:
Diff_iwv_vespaVSmod_scatter = new_mod_vespa.hvplot.scatter(x='IWV_THAAO',
#                                                   y='diff_IWV', by=['time.month'],
#                                                   xlabel='IWV VESPA    [cm]',
#                                                   ylabel='Diff MOD-VESPA    [cm]',
                                                   y='diff_IWV', by=['Time.month'],
                                                   xlabel='IWV VESPA    [mm]',
                                                   ylabel='Diff MOD-VESPA    [mm]',
                                                   height=height, width=width, 
                                                   scale=scale,grid=True)

Diff_iwv_vespaVSmyd_scatter = new_myd_vespa.hvplot.scatter(x='IWV_THAAO',
#                                                   y='diff_IWV', by=['time.month'],
#                                                   xlabel='IWV VESPA    [cm]',
#                                                   ylabel='Diff MYD-VESPA    [cm]',
                                                   y='diff_IWV', by=['Time.month'],
                                                   xlabel='IWV VESPA    [mm]',
                                                   ylabel='Diff MYD-VESPA    [mm]',
                                                   height=height, width=width, 
                                                   scale=scale,grid=True)

(Diff_iwv_vespaVSmod_scatter
+ Diff_iwv_vespaVSmyd_scatter
+ (Diff_iwv_vespaVSmod_scatter*Diff_iwv_vespaVSmyd_scatter
  ).opts(ylabel='Diff MXD-VESPA    [mm]')).cols(1)

In [ ]:
title = f'IWV MXD-VESPA              deltatime={deltatime}             # data points={num_mxd_data_p_Dt}'
((Diff_iwv_vespaVSmod_scatter*Diff_iwv_vespaVSmyd_scatter).
 opts(title=title, ylabel='Diff MXD-VESPA    [mm]'))

### Diff % IWV MXD-THAAO vs IWV THAAO 

In [ ]:
Diffperc_iwv_vespaVSmod_scatter = new_mod_vespa.hvplot.scatter(x='IWV_THAAO',
                                                   y='diff_perc_IWV', by=['Time.month'],
                                                   xlabel='IWV VESPA    [mm]',
                                                   ylabel='100*(MOD-VESPA)/VESPA',
                                                   height=height, width=width, 
                                                   scale=scale,grid=True)

Diffperc_iwv_vespaVSmyd_scatter = new_myd_vespa.hvplot.scatter(x='IWV_THAAO',
                                                   y='diff_perc_IWV', by=['Time.month'],
                                                   xlabel='IWV VESPA    [mm]',
                                                   ylabel='100*(MYD-VESPA)/VESPA',
                                                   height=height, width=width, 
                                                   scale=scale,grid=True)

(Diffperc_iwv_vespaVSmod_scatter
+ Diffperc_iwv_vespaVSmyd_scatter
+ (Diffperc_iwv_vespaVSmod_scatter*Diffperc_iwv_vespaVSmyd_scatter)
 .opts(ylabel='100*(MXD-VESPA)/VESPA')).cols(1)

In [ ]:
title = f'IWV 100*(MXD-VESPA)/VESPA              deltatime={deltatime}             # data points={num_mxd_data_p_Dt}'      # data points={number_MXD_data_points_deltatime}'
((Diffperc_iwv_vespaVSmod_scatter*Diffperc_iwv_vespaVSmyd_scatter)
 .opts(title=title, ylabel='100*(MXD-VESPA)/VESPA'))